# Model construction and modification

`COBREXA` can load models stored in `.mat`, `.json`, and `.xml` formats; and convert
these into `StandardModel`s. However, it is also possible to construct models
from scratch, and modify existing models. This will be demonstrated
here.

In [1]:
using COBREXA

In `COBREXA`, model construction is primarily supported through `StandardModel`s.
To begin, create an empty `StandardModel`.

In [2]:
model = StandardModel("FirstModel") # assign model id = "FirstModel"

Metabolic model of type StandardModel
sparse(Int64[], Int64[], Float64[], 0, 0)
Number of reactions: 0
Number of metabolites: 0


Next, genes, metabolites and reactions need to be added to the model.

### Add genes to the model

In [3]:
gene_list = [Gene(string("g", num)) for num = 1:8]


add_genes!(model, gene_list)

### Add metabolites to the model

In [4]:
metabolite_list = [Metabolite(string("m", num)) for num = 1:4]

metabolite_list[1].formula = "C6H12O6" # can edit metabolites, etc. directly

add_metabolites!(model, metabolite_list)

### Add reactions to the model

There are two ways to create and add reactions to a model.
These are using functions, or macros.

In [5]:
r_m1 = Reaction("EX_m1", Dict("m1" => -1.0), :bidirectional) # exchange reaction: m1 <-> (is the same as m1 ↔ nothing)
r1 = Reaction("r1", Dict("m1" => -1.0, "m2" => 1.0), :forward)
r1.grr = [["g1", "g2"], ["g3"]] # add some gene reaction rules
r2 = Reaction("r2", Dict("m2" => -1.0, "m1" => 1.0), :reverse)
r3 = Reaction("r3", Dict("m2" => -1.0, "m3" => 1.0), :bidirectional)

add_reactions!(model, [r1, r2, r3, r_m1]) # function approach

m1 = metabolite_list[1]
m2 = metabolite_list[2]
m3 = metabolite_list[3]
m4 = metabolite_list[4]

@add_reactions! model begin # macro approach
    "r4", m2 → m4, 0, 1000
    "r_m3", m3 ↔ nothing, -1000, 1000
    "r_m4", m4 → nothing
    "r5", m4 → m2
end

model.reactions["r4"].grr = [["g5"], ["g6", "g7"], ["g8"]]

3-element Vector{Vector{String}}:
 ["g5"]
 ["g6", "g7"]
 ["g8"]

The constructed model can now be inspected.

In [6]:
model

Metabolic model of type StandardModel
sparse([1, 2, 1, 2, 2, 3, 1, 2, 4, 3, 4, 2, 4], [1, 1, 2, 2, 3, 3, 4, 5, 5, 6, 7, 8, 8], [-1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0], 4, 8)
Number of reactions: 8
Number of metabolites: 4


## Modifying existing models

It is also possible to modify a model by deleting certain genes.
This is simply achieved by calling `remove_genes!`.

In [7]:
remove_genes!(model, ["g1", "g2"]; knockout_reactions = false)
model

Metabolic model of type StandardModel
sparse([1, 2, 1, 2, 2, 3, 1, 2, 4, 3, 4, 2, 4], [1, 1, 2, 2, 3, 3, 4, 5, 5, 6, 7, 8, 8], [-1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0], 4, 8)
Number of reactions: 8
Number of metabolites: 4


Likewise, reactions and metabolites can also be deleted.

In [8]:
remove_metabolite!(model, "m1")
model

Metabolic model of type StandardModel
sparse([1, 2, 1, 3, 2, 3, 1, 3], [1, 1, 2, 2, 3, 4, 5, 5], [-1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0], 3, 5)
Number of reactions: 5
Number of metabolites: 3


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*